In [25]:
#%%-------------------------------------------------------------------------------------------
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch

import functions_IMU
import functions_generell
import functions_PRESSURE

In [26]:
#%% Daten einlesen für alle Teilnehmer-------------------------------------------------------------------------------------------
# liste der Teilneher
participants = ['ID_1_Dabisch_Samuel', 'ID_2_Pohl_Jannis', 'ID_3_Kleber_Christian',
                'ID_4_Schröter_Till', 'ID_5_Zaschke_Lenard', 'ID_6_Petroll_Finn', 'ID_7_Gruber_Julius']
# bedingunegn der Messungen
measurements = ['REAL_1', 'REAL_2', 'VR_1', 'VR_2']

# Alle Daten der Teilnehmer einlesen und verarbeiten
all_data = {}

for participant in participants:
    folder_path = os.path.join('data_final', participant)
    
    # Lade die 6 CSV-Dateien für den Teilnehmer
    real_1, real_2, vr_1, vr_2, mvc_beine, mvc_hals = functions_generell.load_csvs(folder_path)
    
    # Speichere die DataFrames im Dictionary
    all_data[participant] = {
        'REAL_1': real_1,
        'REAL_2': real_2,
        'VR_1': vr_1,
        'VR_2': vr_2,
        'MVC_Beine': mvc_beine,
        'MVC_Hals': mvc_hals
    }
    
    print(f"Daten für {participant} geladen")

print(f"\nAlle Daten geladen! {len(all_data)} Teilnehmer erfolgreich eingelesen.")

Daten für ID_1_Dabisch_Samuel geladen
Daten für ID_2_Pohl_Jannis geladen
Daten für ID_3_Kleber_Christian geladen
Daten für ID_4_Schröter_Till geladen
Daten für ID_5_Zaschke_Lenard geladen
Daten für ID_6_Petroll_Finn geladen
Daten für ID_7_Gruber_Julius geladen

Alle Daten geladen! 7 Teilnehmer erfolgreich eingelesen.


## hier werden der takeoff zeitpunkt anhand des Kniegelenks bestimmt

In [27]:
### takeofoff mit kniewinkel
def identify_jumps_by_knee(df, angle_col='RT Knee Flexion (deg)', min_jump_distance_sec=5.0, threshold_angle=60, buffer=0.75, fs=2000):
    """
    Identifiziert Sprünge sequentiell ohne Filterung:
    Sucht nach dem Peak > 60° und nimmt das ALLERERSTE lokale Minimum danach als Takeoff.
    """
    angle = df[angle_col].values
    time = df['time'].values
    
    jumps_list = []
    i = 0
    pause_points = int(min_jump_distance_sec * fs)
    search_window_points = int(1.5 * fs)

    while i < len(angle):
        # A. Suche nach dem nächsten Punkt über dem Schwellenwert (Ausholbewegung)
        if angle[i] >= threshold_angle:
            # 1. Den exakten Peak (maximale Beugung) im Umkreis finden
            peak_search_end = min(i + int(0.5 * fs), len(angle))
            peak_idx = i + np.argmax(angle[i:peak_search_end])
            
            # 2. Suche Takeoff: Das ERSTE lokale Minimum nach dem Peak
            takeoff_idx = peak_idx # Fallback
            
            # Wir laufen vom Peak vorwärts und suchen den ersten Punkt, 
            # an dem die Kurve nicht mehr weiter fällt.
            for j in range(peak_idx + 1, min(peak_idx + search_window_points, len(angle) - 1)):
                if angle[j] <= angle[j-1] and angle[j] <= angle[j+1]:
                    takeoff_idx = j
                    break
            else:
                # Falls kein lokales Minimum gefunden wurde, Backup: Absolutes Minimum
                search_end = min(peak_idx + search_window_points, len(angle))
                takeoff_idx = peak_idx + np.argmin(angle[peak_idx:search_end])

            # 3. Daten extrahieren
            t_absprung = time[takeoff_idx]
            start_analyse = max(0, t_absprung - buffer)
            
            jumps_list.append({
                'sprung nr.': len(jumps_list) + 1,
                'start_analyse': round(start_analyse, 4),
                't_absprung': round(t_absprung, 4)
            })
            
            # 4. Sperrfrist: 5 Sekunden nach dem Takeoff überspringen
            i = takeoff_idx + pause_points
            continue
        
        i += 1
            
    return jumps_list

def visualize_knee_jumps(df, jumps_list, participant="", measurement=""):
    """
    Visualisiert die Sprungerkennung basierend auf dem Kniewinkel.
    Korrektur: alpha in tick_params entfernt, da nicht unterstützt.
    """
    time = df['time'].values
    # Sicherstellen, dass die Spalte exakt so heißt
    knee_angle = df['RT Knee Flexion (deg)'].values
    total_force = df['LT Force (N)'] + df['RT Force (N)']
    
    fig, ax1 = plt.subplots(figsize=(15, 7))

    # Primäre Achse: Kniewinkel
    ax1.plot(time, knee_angle, color='blue', alpha=0.6, label='Kniewinkel (RT Flexion)')
    ax1.set_ylabel('Kniewinkel (deg)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue') # alpha entfernt

    # # Sekundäre Achse für die Kraft
    # ax2 = ax1.twinx()
    # ax2.plot(time, total_force, color='black', alpha=0.2, label='Gesamtkraft (N)')
    # ax2.set_ylabel('Kraft (N)', color='gray') 
    # ax2.tick_params(axis='y', labelcolor='gray') # alpha entfernt

    labeled_buffer = False
    labeled_takeoff = False

    for j in jumps_list:
        # 1. Analyse-Fenster (Buffer bis Takeoff)
        # Beachte: Die Keys müssen 'start_analyse' und 't_absprung' sein (wie in deiner Funktion)
        ax1.axvspan(j['start_analyse'], j['t_absprung'], 
                    color='gray', alpha=0.15, 
                    label='Analyse-Fenster (Buffer)' if not labeled_buffer else "")
        labeled_buffer = True
        
        # 2. Vertikale Linie für Takeoff
        ax1.axvline(j['t_absprung'], color='red', linestyle='--', lw=2,
                    label='Takeoff (Knie-Extremum)' if not labeled_takeoff else "")
        labeled_takeoff = True
        
        # 3. Sprungnummer
        ax1.text(j['t_absprung'], max(knee_angle) * 0.95, f"S{j['sprung nr.']}", 
                 color='red', fontweight='bold', ha='right')

    plt.title(f"Knie-basierte Sprungerkennung: {participant} - {measurement} ({len(jumps_list)} Sprünge)")
    ax1.set_xlabel("Zeit (s)")
    
    # Legenden zusammenführen
    lines1, labels1 = ax1.get_legend_handles_labels()
    # lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 , labels1 , loc='upper right')
    
    ax1.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()

output_folder = 'jump_analysis_results_knee'

# Erstelle Ausgabeverzeichnis, falls nicht vorhanden
os.makedirs(output_folder, exist_ok=True)

for participant in participants:
    print(f"\nVerarbeite {participant}...")
    
    all_jumps_data = []
    
    # Durchlaufe alle 4 Messungen
    for measurement in measurements:
        df = all_data[participant][measurement]
        
        # Identifiziere Sprünge in dieser Messung
        jumps = identify_jumps_by_knee(df,  min_jump_distance_sec = 5, buffer=0.7)
        #visualize_knee_jumps(df, jumps, participant, measurement)

        # Füge Measurement-Info zu jedem Sprung hinzu
        for jump in jumps:
            jump['messung'] = measurement
            all_jumps_data.append(jump)
        
        print(f"  {measurement}: {len(jumps)} Sprünge gefunden")
    
    # Erstelle DataFrame aus allen Sprüngen
    jumps_df = pd.DataFrame(all_jumps_data)
    
    # Speichere als CSV
    output_file = os.path.join(output_folder, f'{participant}_jumps.csv')
    jumps_df.to_csv(output_file, index=False)
    
    print(f"  ✓ Gesamt: {len(all_jumps_data)} Sprünge")
    print(f"  ✓ CSV gespeichert: {output_file}")

print(f"\n{'='*80}")
print(f"Verarbeitung abgeschlossen!")
print(f"Alle CSVs wurden im Ordner '{output_folder}' gespeichert.")
print(f"{'='*80}")



Verarbeite ID_1_Dabisch_Samuel...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_1_Dabisch_Samuel_jumps.csv

Verarbeite ID_2_Pohl_Jannis...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 5 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 1 Sprünge gefunden
  ✓ Gesamt: 18 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_2_Pohl_Jannis_jumps.csv

Verarbeite ID_3_Kleber_Christian...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_3_Kleber_Christian_jumps.csv

Verarbeite ID_4_Schröter_Till...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 5 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 23 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_4_Schröter_Till_jumps.csv

Verarbeite 

## Hier werden die Jump parameter (dauer, Höhe) anhand der Kraft daten bestimmt

In [30]:
### parameter aus Kraftdaten extrahieren
def identify_jumps(df, flight_threshold=50, min_flight_sec=0.2):
    """
    Identifiziert Sprünge über einen Kraft-Schwellenwert und berechnet die Sprunghöhe.
    
    Parameters:
    -----------
    flight_threshold : float
        Kraftwert (N), unter dem eine Flugphase erkannt wird (Default: 50N).
    min_flight_sec : float
        Mindestdauer in der Luft, um als Sprung zu gelten.
    """
    total_force = (df['LT Force (N)'] + df['RT Force (N)']).values
    time = df['time'].values
    g = 9.81 # Erdbeschleunigung
    
    # 1. Flugphasen-Maske
    in_air = total_force < flight_threshold
    
    # 2. Detektion von Statuswechseln
    diff = np.diff(in_air.astype(int))
    takeoff_indices = np.where(diff == 1)[0]
    landing_indices = np.where(diff == -1)[0]
    
    # Korrektur der Indizes (Start mit Take-off, Ende mit Landung)
    if len(takeoff_indices) > 0 and len(landing_indices) > 0:
        if landing_indices[0] < takeoff_indices[0]:
            landing_indices = landing_indices[1:]
        takeoff_indices = takeoff_indices[:len(landing_indices)]

    jumps_list = []
    
    # 3. Flugdaten extrahieren und Höhe berechnen
    for i in range(len(takeoff_indices)):
        idx_off = takeoff_indices[i]
        idx_on = landing_indices[i]
        
        t_off = time[idx_off]
        t_on = time[idx_on]
        flug_dauer = t_on - t_off
        
        # Filter für plausible Sprünge
        if flug_dauer > min_flight_sec:
            # Berechnung der Sprunghöhe in Metern
            # Formel: h = 1/8 * g * t^2
            sprunghoehe_m = (1/8) * g * (flug_dauer**2)
            sprunghoehe_cm = sprunghoehe_m * 100
            
            jump_dict = {
                'sprung nr.': len(jumps_list) + 1,
                't_absprung': round(t_off, 4),
                't_landung': round(t_on, 4),
                'flugzeit_s': round(flug_dauer, 4),
                'sprunghoehe_cm': round(sprunghoehe_cm, 2)
            }
            jumps_list.append(jump_dict)
            
    return jumps_list

def visualize_jumps(df, jumps_list, participant, measurement):
    # Kraft berechnen
    total_force = df['LT Force (N)'] + df['RT Force (N)']
    time = df['time'].values
    
    plt.figure(figsize=(15, 7))
    plt.plot(time, total_force, color='black', alpha=0.3, label='Rohdaten Kraft')
    
    # Hilfsvariablen für die Legende
    labeled_flight_zone = False
    labeled_events = False
    
    for j in jumps_list:
        # 1. Die Flugphase (t_absprung bis t_landung) markieren
        plt.axvspan(j['t_absprung'], j['t_landung'], 
                    color='orange', alpha=0.2, 
                    label='Flugphase' if not labeled_flight_zone else "")
        labeled_flight_zone = True
        
        # 2. Vertikale Linien für Take-off und Landing
        plt.axvline(j['t_absprung'], color='green', linestyle='--', alpha=0.6, 
                    label='Take-off' if not labeled_events else "")
        plt.axvline(j['t_landung'], color='red', linestyle='--', alpha=0.6, 
                    label='Landing' if not labeled_events else "")
        labeled_events = True
        
        # 3. Text-Position (Mitte der Flugphase)
        text_pos_x = j['t_absprung'] + (j['flugzeit_s'] / 2)
        
        # 4. Sprungnummer und Sprunghöhe anzeigen
        # Wir platzieren den Text etwas über der Kraftkurve
        y_pos = max(total_force) * 0.9
        plt.text(text_pos_x, y_pos, f"Sprung {j['sprung nr.']}\n{j['sprunghoehe_cm']} cm", 
                 horizontalalignment='center', fontweight='bold', fontsize=10,
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='orange', boxstyle='round,pad=0.5'))
        
        # 5. Flugzeit darunter anzeigen
        plt.text(text_pos_x, y_pos * 0.82, f"t: {j['flugzeit_s']}s", 
                 horizontalalignment='center', fontsize=9, color='black', alpha=0.7)

    # Titel und Labels
    title_str = f"Sprungdetektion: {participant} - {measurement} | {len(jumps_list)} Sprünge gefunden"
    plt.title(title_str, fontsize=14, pad=15)
    plt.xlabel("Zeit (s)")
    plt.ylabel("Gesamtkraft (N)")
    
    # Verzeichnis erstellen (falls nicht vorhanden)
    save_dir = 'Pictures_Test/Sprungzyklen_Kraft'
    os.makedirs(save_dir, exist_ok=True)
    
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.2)
    plt.tight_layout()
    
    # Speichern
    #file_name = f'{participant}_{measurement}_jump_analysis.png'
    #plt.savefig(os.path.join(save_dir, file_name), dpi=300)

    plt.show()


output_folder = 'jump_analysis_results_Pressure'

# Erstelle Ausgabeverzeichnis, falls nicht vorhanden
os.makedirs(output_folder, exist_ok=True)

for participant in participants:
    print(f"\nVerarbeite {participant}...")
    
    all_jumps_data = []
    
    # Durchlaufe alle 4 Messungen
    for measurement in measurements:
        df = all_data[participant][measurement]
        
        # Identifiziere Sprünge in dieser Messung
        jumps = identify_jumps(df, flight_threshold=155, min_flight_sec=0.2)
        #visualize_jumps(df, jumps, participant, measurement)

        # Füge Measurement-Info zu jedem Sprung hinzu
        for jump in jumps:
            jump['messung'] = measurement
            all_jumps_data.append(jump)
        
        print(f"  {measurement}: {len(jumps)} Sprünge gefunden")
    
    # Erstelle DataFrame aus allen Sprüngen
    jumps_df = pd.DataFrame(all_jumps_data)
    
    # Speichere als CSV
    output_file = os.path.join(output_folder, f'{participant}_jumps.csv')
    jumps_df.to_csv(output_file, index=False)
    
    print(f"  ✓ Gesamt: {len(all_jumps_data)} Sprünge")
    print(f"  ✓ CSV gespeichert: {output_file}")

print(f"\n{'='*80}")
print(f"Verarbeitung abgeschlossen!")
print(f"Alle CSVs wurden im Ordner '{output_folder}' gespeichert.")
print(f"{'='*80}")



Verarbeite ID_1_Dabisch_Samuel...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_1_Dabisch_Samuel_jumps.csv

Verarbeite ID_2_Pohl_Jannis...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 5 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 2 Sprünge gefunden
  ✓ Gesamt: 19 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_2_Pohl_Jannis_jumps.csv

Verarbeite ID_3_Kleber_Christian...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_3_Kleber_Christian_jumps.csv

Verarbeite ID_4_Schröter_Till...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_4_Schröter_Till_jumps.

## sprünge nach der Höhe sortieren

## Zeitnormieren auf sekunde vor takeoff

In [59]:
### Zeitnormerierung der Sprünge basierend auf Knie-Extraktion
def time_normalize_jumps_knee(df, participant_name, measurement_type, 
                             jumps_csv_folder='jump_analysis_results_knee', 
                             normalize_points=100):
    """
    Normiert die Sprünge basierend auf der Knie-Extraktion (start_analyse bis t_absprung).
    
    Parameters:
    -----------
    df : pd.DataFrame
        Der Rohdaten-DataFrame (z.B. df_real1).
    participant_name : str
        Name des Probanden (für den CSV-Dateinamen).
    measurement_type : str
        Die aktuelle Messung (z.B. 'REAL_1').
    jumps_csv_folder : str
        Ordner, in dem die Ergebnisse der Knie-Analyse liegen.
    normalize_points : int
        Anzahl der Datenpunkte nach der Normierung (Standard: 100).
    """
    
    # Pfad zur CSV (jetzt aus dem knee-Ordner)
    jumps_csv_path = os.path.join(jumps_csv_folder, f'{participant_name}_jumps.csv')
    
    if not os.path.exists(jumps_csv_path):
        print(f"Warnung: CSV nicht gefunden {jumps_csv_path}")
        return {}
    
    # 1. CSV laden
    jumps_info = pd.read_csv(jumps_csv_path)
    
    # 2. Filter auf die aktuelle Messung (z.B. nur REAL_1)
    relevant_jumps = jumps_info[jumps_info['messung'] == measurement_type]
    
    normalized_jumps = {}
    
    # 3. Über die gefundenen Sprünge iterieren
    for idx, row in relevant_jumps.iterrows():
        # Zeitgrenzen aus deiner neuen Struktur
        start_time = row['start_analyse']
        end_time = row['t_absprung']
        jump_nr = int(row['sprung nr.'])
        
        # Daten im Zeitfenster ausschneiden
        mask = (df['time'] >= start_time) & (df['time'] <= end_time)
        jump_data = df[mask].reset_index(drop=True)
        
        # Sicherheitscheck falls keine Daten im Fenster sind
        if len(jump_data) < 2:
            continue
            
        # --- Zeit-Normierung (Interpolation) ---
        # Erstelle Indizes für die ursprünglichen Daten (z.B. 0 bis 145)
        original_indices = np.linspace(0, len(jump_data) - 1, len(jump_data))
        # Erstelle Indizes für die Ziel-Punkte (z.B. 0 bis 99)
        new_indices = np.linspace(0, len(jump_data) - 1, normalize_points)
        
        normalized_data = {}
        for col in jump_data.columns:
            # Nur numerische Spalten (Kraft, Winkel, EMG) interpolieren
            if pd.api.types.is_numeric_dtype(jump_data[col]):
                normalized_data[col] = np.interp(new_indices, original_indices, jump_data[col].values)
            else:
                # Nicht-numerische Spalten (z.B. ID) einfach den ersten Wert beibehalten
                normalized_data[col] = [jump_data[col].iloc[0]] * normalize_points
        
        # Füge die normierte Zeitachse (0% bis 100%) hinzu
        normalized_data['time_normalized'] = np.linspace(0, 1, normalize_points)
        
        # In DataFrame umwandeln
        normalized_df = pd.DataFrame(normalized_data)
        
        # Eindeutiger Key für das Dictionary
        key = f"{measurement_type}_jump_{jump_nr}"
        normalized_jumps[key] = normalized_df
    
    return normalized_jumps

# %% Zeit-Normierung basierend auf Knie-Extraktion
print(f"\nStarte Zeit-Normierung (Knie-Logik)...")

# Neues Zielverzeichnis für die Knie-Ergebnisse
output_folder_knee = 'jump_analysis_results_knee'

all_normalized_data = {}

for participant in participants:
    print(f"Normiere Daten für {participant}...")
    all_normalized_data[participant] = {}
    
    for measurement in measurements:
        df = all_data[participant][measurement]
        
        # Wichtig: Nutze hier die neue Funktion, die 'start_analyse' und 't_absprung' liest
        norm_jumps_dict = time_normalize_jumps_knee(
            df, 
            participant, 
            measurement, 
            jumps_csv_folder=output_folder_knee, # Geänderter Ordner
            normalize_points=100
        )
        
        # Speichern im Haupt-Dictionary
        all_normalized_data[participant].update(norm_jumps_dict)
        
        print(f"  ✓ {measurement}: {len(norm_jumps_dict)} Sprünge normiert")

print(f"\nFertig! Alle Sprünge sind nun zeitnormiert (Buffer -> Take-off) verfügbar.")


Starte Zeit-Normierung (Knie-Logik)...
Normiere Daten für ID_1_Dabisch_Samuel...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 6 Sprünge normiert
  ✓ VR_1: 6 Sprünge normiert
  ✓ VR_2: 6 Sprünge normiert
Normiere Daten für ID_2_Pohl_Jannis...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 5 Sprünge normiert
  ✓ VR_1: 6 Sprünge normiert
  ✓ VR_2: 1 Sprünge normiert
Normiere Daten für ID_3_Kleber_Christian...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 6 Sprünge normiert
  ✓ VR_1: 6 Sprünge normiert
  ✓ VR_2: 6 Sprünge normiert
Normiere Daten für ID_4_Schröter_Till...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 6 Sprünge normiert
  ✓ VR_1: 5 Sprünge normiert
  ✓ VR_2: 6 Sprünge normiert
Normiere Daten für ID_5_Zaschke_Lenard...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 6 Sprünge normiert
  ✓ VR_1: 6 Sprünge normiert
  ✓ VR_2: 6 Sprünge normiert
Normiere Daten für ID_6_Petroll_Finn...
  ✓ REAL_1: 6 Sprünge normiert
  ✓ REAL_2: 6 Sprünge normiert
  ✓ VR_1: 6 Sprünge normiert
  ✓ VR_2: 6 Sprünge n

## Knie, Hüft und Sprunggelenk kurven pro teilneher und über alle teilnehmer + mittelwert/ standartabweichung 

In [60]:
### Plotten und speichern der normalisierten Bedingungen
def plot_normalized_conditions(normalized_dict, participant_name, condition_type, column_to_plot):
    """
    Fasst alle Sprünge einer Bedingung (z.B. REAL_1 + REAL_2) zusammen.
    Erstellt:
    1. Ein großes Subplot-Bild mit allen ~12 Sprüngen.
    2. Ein Vergleichsbild (Overlaid) mit dem Gesamt-Mittelwert der Bedingung.
    """
    # 1. Vorbereitung Ordnerstruktur
    target_dir = os.path.join('Pictures_Test', 'Conditions_Combined', column_to_plot.replace(' ', '_'))
    os.makedirs(target_dir, exist_ok=True)
    
    # Filtere die Keys: Sucht nach allen Keys, die den Bedingungs-String enthalten
    # z.B. wenn condition_type='REAL', findet er 'REAL_1_jump_1' bis 'REAL_2_jump_6'
    jump_keys = [k for k in normalized_dict.keys() if condition_type in k]
    num_jumps = len(jump_keys)
    
    if num_jumps == 0:
        print(f"Keine Daten für Bedingung {condition_type} bei {participant_name} gefunden.")
        return

    # Daten sammeln
    all_values = []
    time_normalized = None

    # --- GRAFIK 1: SUBPLOTS (Alle 12 untereinander) ---
    # Wir machen das Layout etwas kompakter, da 12 Sprünge viel Platz brauchen
    fig_sub, axes = plt.subplots(num_jumps, 1, figsize=(10, 2 * num_jumps), sharex=True)
    if num_jumps == 1: axes = [axes]
    
    for i, key in enumerate(jump_keys):
        df_jump = normalized_dict[key]
        val = df_jump[column_to_plot].values
        all_values.append(val)
        if time_normalized is None:
            time_normalized = df_jump['time_normalized'].values * 100 # In Prozent umrechnen
            
        axes[i].plot(time_normalized, val, color='black', lw=1)
        axes[i].set_ylabel("Wert", fontsize=8)
        axes[i].set_title(f"{key}", fontsize=9, pad=2)
        axes[i].grid(True, alpha=0.2)
    
    plt.xlabel("Zeitverlauf Normiert (0-100% vor Takeoff)")
    plt.tight_layout()
    fig_sub.savefig(os.path.join(target_dir, f"{participant_name}_{condition_type}_all_subplots.png"))
    plt.close(fig_sub)

    # --- GRAFIK 2: ÜBEREINANDERGELEGT (12 Sprünge + 1 dicker Mittelwert) ---
    plt.figure(figsize=(11, 7))
    
    data_matrix = np.array(all_values)
    mean_val = np.mean(data_matrix, axis=0)
    std_val = np.std(data_matrix, axis=0)

    # Einzelne Sprünge (sehr blass im Hintergrund)
    for i, val in enumerate(all_values):
        plt.plot(time_normalized, val, alpha=0.15, lw=1, color='gray')
    
    # Mittelwert (Fett in Farbe je nach Bedingung)
    color = 'blue' if 'REAL' in condition_type else 'red'
    plt.plot(time_normalized, mean_val, color=color, lw=4, label=f'Gesamt-Mittelwert {condition_type} (n={num_jumps})')
    
    # Standardabweichung (Schatten)
    plt.fill_between(time_normalized, mean_val - std_val, mean_val + std_val, 
                     color=color, alpha=0.2, label=f'Standardabweichung (±1 SD)')
    
    plt.title(f"Bedingungs-Analyse: {participant_name}\nVariable: {column_to_plot} | Bedingung: {condition_type}", fontsize=12)
    plt.xlabel("Vorbereitungsphase in % (0% = Start, 100% = Takeoff)")
    plt.ylabel(column_to_plot)
    plt.legend(loc='upper left', frameon=True)
    plt.grid(True, alpha=0.3)
    
    # Speichern Overlaid
    plt.savefig(os.path.join(target_dir, f"{participant_name}_{condition_type}_combined_mean.png"), dpi=300)
    plt.close()

    #print(f"  ✓ Kombinierte {condition_type}-Grafiken (n={num_jumps}) gespeichert in: {target_dir}")

### Plot funkton für einen Winkel
def plot_everything_for_one_column_to_plot (column_to_plot):
    """
    Wrapper-Funktion, um für alle Teilnehmer und eine bestimmte Spalte die 
    kombinierten Plots zu erstellen.
    """
    print(f"\nStarte Generierung der kombinierten Plots (12 Sprünge pro Bedingung)...")

    for participant in participants:
        #print(f"\nErstelle kombinierte Plots für {participant} - Variable: {column_to_plot}")
        
        # Bedingungen REAL und VR
        for condition in ['REAL', 'VR']:
            plot_normalized_conditions(
                all_normalized_data[participant], 
                participant, 
                condition, 
                column_to_plot
            )

        # print(f"  ✓ Bedingungen REAL und VR für {participant} erfolgreich zusammengefasst.")
    
    print(f"\n{'='*30}")
    print("ALLE KOMBINIERTEN GRAFIKEN ERSTELLT")
    print(f"Speicherort: Pictures_Test/Conditions_Combined/{column_to_plot.replace(' ', '_').replace('(', '').replace(')', '')}/")
    print(f"{'='*30}")  

### globaler plot für alle Teilnehmer einer Bedingung
def plot_global_condition_comparison(all_participants_dict, condition_type, column_to_plot):
    """
    Erstellt einen Plot für ALLE Teilnehmer zusammengefasst für eine Bedingung (REAL oder VR).
    - Dünne graue Linien: Jeder einzelne Sprung (n=~84)
    - Dicke farbige Linie: Gesamt-Mittelwert über alle Teilnehmer
    - Schattierter Bereich: Standardabweichung
    """
    
    # 1. Vorbereitung Ordnerstruktur
    target_dir = os.path.join('Pictures_Test', 'Global_Comparison', column_to_plot.replace(' ', '_'))
    os.makedirs(target_dir, exist_ok=True)
    
    all_values = []
    time_normalized = None
    total_jumps_count = 0

    # 2. Daten sammeln über alle Teilnehmer hinweg
    for p_name, normalized_dict in all_participants_dict.items():
        # Suche Sprünge für die Bedingung (z.B. REAL) bei diesem Teilnehmer
        jump_keys = [k for k in normalized_dict.keys() if condition_type in k]
        
        for key in jump_keys:
            df_jump = normalized_dict[key]
            val = df_jump[column_to_plot].values
            all_values.append(val)
            
            if time_normalized is None:
                # In Prozent umrechnen (0-100)
                time_normalized = df_jump['time_normalized'].values * 100
            
            total_jumps_count += 1

    if total_jumps_count == 0:
        print(f"Keine Daten für {condition_type} in der gesamten Gruppe gefunden.")
        return

    # 3. Statistik berechnen
    data_matrix = np.array(all_values)
    mean_val = np.mean(data_matrix, axis=0)
    std_val = np.std(data_matrix, axis=0)

    # 4. Plotting
    plt.figure(figsize=(12, 8))
    
    # Einzelne Sprünge (84 dünne Linien)
    for val in all_values:
        plt.plot(time_normalized, val, alpha=0.08, lw=0.8, color='gray') # Sehr blass für die Masse
    
    # Farbe festlegen
    main_color = 'blue' if 'REAL' in condition_type else 'red'
    
    # Standardabweichung (Schatten)
    plt.fill_between(time_normalized, mean_val - std_val, mean_val + std_val, 
                     color=main_color, alpha=0.15, label=f'SD (±1 σ)')
    
    # Mittelwert (Fett)
    plt.plot(time_normalized, mean_val, color=main_color, lw=3.5, 
             label=f'Gesamt-Mittelwert {condition_type} (n={total_jumps_count} Sprünge)')
    
    # Layout & Beschriftung
    plt.title(f"Globale Analyse: Alle Teilnehmer\nVariable: {column_to_plot} | Bedingung: {condition_type}", fontsize=14)
    plt.xlabel("Bewegungszyklus in % (0% = Start, 100% = Takeoff)", fontsize=12)
    plt.ylabel(column_to_plot, fontsize=12)
    plt.xlim(0, 100)
    plt.legend(loc='best', frameon=True)
    plt.grid(True, linestyle='--', alpha=0.5)
    
    # Speichern
    file_name = f"GLOBAL_{condition_type}_{column_to_plot.replace(' ', '_')}.png"
    plt.savefig(os.path.join(target_dir, file_name), dpi=300, bbox_inches='tight')
    plt.close()

    print(f"  ✓ Globaler Plot gespeichert: {file_name} (n={total_jumps_count} Sprünge)")


### Gelenke pro Teilnehmer:

In [56]:
# plots für Knie aufrufen
colum_to_Plot = 'RT Knee Flexion (deg)'
plot_everything_for_one_column_to_plot(colum_to_Plot)


Starte Generierung der kombinierten Plots (12 Sprünge pro Bedingung)...

ALLE KOMBINIERTEN GRAFIKEN ERSTELLT
Speicherort: Pictures_Test/Conditions_Combined/RT_Knee_Flexion_deg/


In [ ]:
# plots für Hüfte aufrufen
colum_to_Plot = 'RT Hip Flexion (deg)'
plot_everything_for_one_column_to_plot(colum_to_Plot)

In [57]:
# plots für Sprunggelenk aufrufen
colum_to_Plot = 'RT Ankle Dorsiflexion (deg)'
plot_everything_for_one_column_to_plot(colum_to_Plot)



Starte Generierung der kombinierten Plots (12 Sprünge pro Bedingung)...

ALLE KOMBINIERTEN GRAFIKEN ERSTELLT
Speicherort: Pictures_Test/Conditions_Combined/RT_Ankle_Dorsiflexion_deg/


### Globaler Plot mit allen Teilnehmern zusammen

In [61]:
### globale plots für Knie vergleichen
colum_to_Plot = 'RT Knee Flexion (deg)'
plot_global_condition_comparison(all_data, 'REAL', colum_to_Plot)
plot_global_condition_comparison(all_data, 'VR', colum_to_Plot)

KeyError: 'time_normalized'

## Parameter Bestimmen

### Maximaler Winkel innerhalb der Fenster

In [21]:
### Parameter extrahieren
def extract_max_parameter(normalized_dict, column_to_process):
    """
    Extrahiert den Maximalwert einer Spalte für jeden Sprung.
    Gibt ein Dictionary zurück: {'REAL_1_jump_1': 65.4, ...}
    """
    results = {}
    
    for jump_key, df_jump in normalized_dict.items():
        if column_to_process in df_jump.columns:
            max_val = df_jump[column_to_process].max()
            results[jump_key] = round(max_val, 2)
    
    type = 'Maximalwert'
    return results, type

### Parameter speichern/aktualisieren
def update_participant_parameters(participant_name, new_data_dict, column_label, type):
    """
    Speichert Parameter in einer CSV. 
    Garantiert korrekte Spaltenbenennung und verhindert KeyErrors beim Mergen.
    """
    folder = 'Parameter_IMU'
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f'{participant_name}_parameter_{type}.csv')
    
    # 1. Neuen DataFrame aus dem Dictionary erstellen
    rows = []
    for jump_key, value in new_data_dict.items():
        if '_jump_' in jump_key:
            parts = jump_key.split('_jump_')
            trial = parts[0]
            sprung_nr = int(parts[1]) # Als Zahl für saubere Sortierung
        else:
            trial, sprung_nr = jump_key, 0
        
        rows.append({
            'participant': participant_name,
            'trial': trial,
            'sprung_nr': sprung_nr,
            column_label: value
        })
    
    new_df = pd.DataFrame(rows)
    
    # 2. Falls Datei existiert, einlesen und mergen
    if os.path.exists(file_path):
        try:
            existing_df = pd.read_csv(file_path, sep=';')
            
            # Sicherheitscheck: Falls 'participant' fehlt, wurde die CSV falsch erstellt
            if 'participant' not in existing_df.columns:
                print(f"  ! CSV von {participant_name} war fehlerhaft. Wird neu erstellt.")
                new_df.to_csv(file_path, sep=';', index=False)
                return

            # Falls die Spalte schon existiert, im alten DF löschen (für das Update)
            if column_label in existing_df.columns:
                existing_df = existing_df.drop(columns=[column_label])
            
            # Mergen (Zusammenführen)
            updated_df = pd.merge(
                existing_df, 
                new_df, 
                on=['participant', 'trial', 'sprung_nr'], 
                how='outer'
            )
            
            # Sortieren (Real_1 vor Real_2, Sprung 1 vor Sprung 2)
            updated_df = updated_df.sort_values(by=['trial', 'sprung_nr'])
            updated_df.to_csv(file_path, sep=';', index=False)
            
        except Exception as e:
            print(f"  ! Fehler beim Update von {participant_name}: {e}. Erstelle Datei neu.")
            new_df.to_csv(file_path, sep=';', index=False)
    else:
        # 3. Datei existiert noch nicht: Neu erstellen
        new_df.to_csv(file_path, sep=';', index=False)

    print(f"  ✓ Spalte '{column_label}' erfolgreich aktualisiert.")


# %% Parameter-Extraktion (Maximalwerte) --------------------------------------
print(f"\nStarte Extraktion der Maximalwerte...")

# Hier definierst du, welche Spalten du auswerten willst
# Der Key ist die Spalte im DF, der Value ist der Name für deine CSV
columns_to_analyze = {
    'RT Knee Flexion (deg)': 'RT_Knee_Max',
    'LT Knee Flexion (deg)': 'LT_Knee_Max',
    'RT Hip Flexion (deg)': 'RT_Hip_Max',
    'LT Hip Flexion (deg)': 'LT_Hip_Max',
    'RT Ankle Dorsiflexion (deg)': 'RT_Ankle_Max',
    'LT Ankle Dorsiflexion (deg)': 'LT_Ankle_Max'
}

for participant in participants:
    print(f"Verarbeite Parameter für {participant}:")
    
    for raw_col, csv_label in columns_to_analyze.items():
        # 1. Maxima berechnen
        max_values, type  = extract_max_parameter(all_normalized_data[participant], raw_col)
        
        # 2. CSV updaten/erstellen
        if max_values: # Nur wenn Daten gefunden wurden
            update_participant_parameters(participant, max_values, csv_label, type)

print(f"\n{'='*30}")
print(f"ALLE PARAMETER GESPEICHERT IM ORDNER 'Parameter_IMU'")
print(f"{'='*30}")



Starte Extraktion der Maximalwerte...
Verarbeite Parameter für ID_1_Dabisch_Samuel:
  ✓ Spalte 'RT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'RT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte 'RT_Ankle_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Ankle_Max' erfolgreich aktualisiert.
Verarbeite Parameter für ID_2_Pohl_Jannis:
  ✓ Spalte 'RT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'RT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte 'RT_Ankle_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Ankle_Max' erfolgreich aktualisiert.
Verarbeite Parameter für ID_3_Kleber_Christian:
  ✓ Spalte 'RT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Knee_Max' erfolgreich aktualisiert.
  ✓ Spalte 'RT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte 'LT_Hip_Max' erfolgreich aktualisiert.
  ✓ Spalte